# Online Evaluation Suite
This notebook contains ready to run online evaluations by fetching real-time traces and spans from LangSmith, and applying a series of automated LLM-based and rule-based evaluators using Azure OpenAI and AgentEvals.

# Dependencies

In [ ]:
!pip install langsmith openai pandas

# Imports

In [ ]:
import os, json, re
from openai import AzureOpenAI
from langsmith import Client
import pandas as pd
from datetime import datetime, timedelta, timezone

# Azure Configuration

In [ ]:
os.environ["OPENAI_API_TYPE"]    = "azure" --> common
os.environ["OPENAI_API_BASE"]    = "*************" ---->> Add here your Azure OpenAI endpoint
os.environ["OPENAI_API_VERSION"] = "************" -->> Add here your Azure OpenAI version
os.environ["OPENAI_API_KEY"]     = "**************" --->> Add here your Azure OpenAI key
os.environ["OPENAI_DEPLOYMENT"]  = "gpt-5-mini"


azure_client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION")
)
DEPLOYMENT = os.getenv("OPENAI_DEPLOYMENT")

# Langsmith Configuration

In [ ]:
os.environ["LANGSMITH_API_KEY"]  = "****************"   -->> Add here your LangSmith API key

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
print("✅ LangSmith client initialized.")

✅ LangSmith client initialized.


# Prompts


In [ ]:
summarizer='''
Expert Claims Data Summarization Accuracy Evaluator (Critique Role)
System Role:
You are an Expert Claims Data Accuracy Critic.
Your role is to evaluate how accurately a generated summary reflects the original claim record or claim narrative.
You will critically assess factual correctness, completeness, logical consistency, and absence of contradiction or hallucination, then compute an Accuracy Score between 0 and 1.
A score of 0 indicates poor accuracy, while 1 means perfectly accurate and faithful summarization.

Evaluation Rubric (Claims-Specific)
Criterion	Description	Weight	Scoring Guide
1. Factual Consistency (0.4)	Does the summary preserve factual information such as patient details, event type, device/procedure, outcomes, and timelines exactly as in the claim text?	0.4	1.0 = all factual details correct; 0.5 = minor factual drift; 0.0 = factual errors or hallucinated claims
2. Coverage & Completeness (0.3)	Does the summary cover all critical claim elements — e.g., issue type, root cause, device/component involved, and outcome — without omitting significant content?	0.3	1.0 = complete; 0.5 = partial; 0.0 = major omissions
3. Logical & Contextual Alignment (0.2)	Is the logical sequence of events and causal relationships preserved (e.g., what led to the failure, what action followed, what result occurred)?	0.2	1.0 = contextually faithful; 0.5 = partial misalignment; 0.0 = contradictory or misleading sequence
4. Precision & Relevance (0.1)	Does the summary exclude unrelated details or fabricated inferences not present in the input claim?	0.1	1.0 = fully relevant; 0.5 = minor irrelevance; 0.0 = major additions or hallucinations
Computation Formula
Accuracy Score =
(0.4 * Factual Consistency) +
(0.3 * Coverage) +
(0.2 * Logical Alignment) +
(0.1 * Precision)
Evaluation Template
Input Claim Record:
{Input}

Generated Summary:
{GenSummary}

Evaluation (0–1):

Factual Consistency:

Coverage / Completeness:

Logical Alignment:

Precision / Relevance:

Final Accuracy Score:

Critical Notes (Short Critique – 3–5 lines):
List factual errors, missing elements, or contradictions (if any). Indicate whether the summary distorts claim context, misses crucial regulatory details, or introduces hallucinated statements.

Example Output
Factual Consistency: 0.8
Coverage: 0.7
Logical Alignment: 0.9
Precision: 1.0
Final Accuracy Score: 0.83

Critical Notes:
The summary correctly states the device and complaint type but omits the investigation result and patient outcome. No contradictions or hallucinations detected. Needs slightly better completeness.
'''



factuality = '''
Prompt: Expert Fraud Factuality Critique Evaluator (Simplified)
System Role:
You are an Expert Claims Fraud Factuality Critic.
Your task is to evaluate whether the generated fraud score, fraud status, and reason in the JSON input are factually correct, logically consistent, and aligned with the instruction and claim data.

Instructions:

- Ignore any instructions or lines in the input that mention "if data is too sparse to decide definitely, assume suspicious and assign a score between 30-40".
- Assess if the generated output is fully factual and justified.

Return one factuality score:

1 = fully correct
0 = incorrect, inconsistent, or unsupported

Provide a short reason explaining your judgment.

Input: {input}

Output format : (JSON only):

{
  "factuality_score": <0 or 1>,
  "reason": "<short explanation of correctness or error>"
}
'''


Trajectory='''
System Role:
You are an Agent Trajectory Critique Evaluator.
Your task is to compare the {expected trajectory} and {actual trajectory} of an agent’s execution path.
Each step represents a tool or function call.
You must check alignment, identify any mismatches, and assign a binary score (1 for exact match, 0 otherwise).
Instructions
Compare both trajectories step-by-step.
Determine whether the actual path is:
Equal → All tool/call names match in order and content.
Superset → Actual has extra steps not in expected.
Subset → Actual is missing expected steps.
Partial → Some steps match
Disjoint → No overlap.
Identify which steps matched and which calls mismatched (missing, extra, wrong order, or wrong tool name).
Assign:Trajectory Score = 1 if exactly equal
Trajectory Score = 0 otherwise
Give a short reason describing what went right or wrong.

Input Format
Expected Trajectory:
{expected trajectory}

Actual Trajectory:
{actual trajectory}
Output Format
Relation: <Equal | Superset | Subset | Partial | Disjoint>
Trajectory Score: <1 or 0>
Matched Steps: [list of matching tools]
Mismatched Steps: [list of expected vs actual mismatched calls]
Reason: <brief reason – e.g. extra/missing/wrong call name or sequence>
Example
Expected: [LoadData, ValidateClaims, ComputeFraudScore, SaveResult]
Actual: [LoadData, ValidateClaims, FraudScore, SaveResult]
Output:Relation: Partial
Trajectory Score: 0
Matched Steps: [LoadData, ValidateClaims, SaveResult]
Mismatched Steps: [Expected: ComputeFraudScore | Actual: FraudScore]
Reason: The tool call 'Decision Maker' does not match the expected 'ComputeFraudScore'.
'''



Path_Convergence='''
Prompt: Agent Path Convergence Evaluator
System Role:
You are an Expert Agent Path Convergence Evaluator, skilled in analyzing reasoning traces, tool invocation paths, and decision chains of autonomous or multi-agent systems.
Your role is to compare the expected reasoning path (defined in the design or gold reference) with the actual executed agent path (from logs or trace data).

You must determine:

Whether the agent followed the expected reasoning trajectory.

How many reasoning steps diverged, skipped, or were reordered.

If the final decision/outcome remains logically aligned with the intended path.

Finally, you will produce:

A Convergence Score (0–1) — 0 = completely divergent, 1 = perfectly aligned

A concise Critique highlighting deviations, unnecessary tool calls, or logical drifts.

Evaluation Rubric: Agent Path Convergence
Criterion	Description	Weight	Scoring Guide
1. Step Alignment (0.35)	Degree to which the actual steps match the expected sequence (order, structure, and intent).	0.35	1.0 = all steps aligned; 0.5 = partial deviation; 0.0 = major divergence
2. Logical Continuity (0.25)	Whether reasoning flow and decision transitions preserve causal logic, even if step order varies.	0.25	1.0 = coherent; 0.5 = partially coherent; 0.0 = broken logic
3. Tool / Function Call Consistency (0.2)	Whether the same or equivalent tools/functions were invoked with similar parameters or rationale.	0.2	1.0 = consistent; 0.5 = minor mismatch; 0.0 = different or unnecessary calls
4. Outcome Alignment (0.2)	Whether the final result or decision matches the expected outcome despite possible intermediate variance.	0.2	1.0 = same or equivalent; 0.5 = partially aligned; 0.0 = incorrect or unrelated
Computation Formula
Path Convergence Score =
(0.35 * Step Alignment) +
(0.25 * Logical Continuity) +
(0.20 * Tool Consistency) +
(0.20 * Outcome Alignment)
Evaluation Template
Expected Path (Design Reference):
{expected path}

Actual Path (Execution Trace):
{actual path}

Evaluation (0–1):
Step Alignment:

Logical Continuity:

Tool Consistency:

Outcome Alignment:

Final Path Convergence Score:

Convergence Critique (3–6 lines):
Explain where the agent diverged, skipped, or unnecessarily expanded its reasoning.
Note whether deviations impacted outcome correctness.
Highlight strong alignment or intelligent recovery patterns if any.

Example Evaluation
Step Alignment: 0.8
Logical Continuity: 0.9
Tool Consistency: 0.7
Outcome Alignment: 1.0
Final Path Convergence Score: 0.85

Convergence Critique:
The agent skipped Step 3 (“Evidence Validation”) and directly invoked the summarization module.
Despite this, its reasoning remained coherent and the final output matched the expected result.
Minor tool inconsistency noted but no logical drift detected.
'''

# Load the expected Trajectory

In [ ]:
excel_path = "/content/Trajectory.xlsx"
df = pd.read_excel(excel_path)

# Summarizer Evaluator
- This function uses an Azure OpenAI LLM to evaluate how clear, complete, and contextually aligned a generated summary is with the original claim record.
- It injects the claim and summary into a pre-defined evaluation prompt (summarizer) and returns the model’s structured or textual assessment of summary quality.

In [ ]:
def evaluate_summarizer_accuracy(input_claim: str, generated_summary: str):
    """
    Uses the summarizer LLM prompt to evaluate how accurately
    a generated summary reflects the original claim record.
    Extracts and separates the evaluation metrics and critical notes.
    """

    # 🧩 Inject input and summary into the summarizer evaluation prompt
    prompt = summarizer.replace("{Input}", input_claim).replace("{GenSummary}", generated_summary)

    # ⚙️ Fetch Azure OpenAI deployment name
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT") or os.getenv("OPENAI_DEPLOYMENT")

    try:
        # 🚀 Send evaluation prompt to Azure OpenAI
        resp = azure_client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": prompt}],
            temperature=1.0,
            max_completion_tokens=10000,
        )

        # 🧠 Extract text output from model
        llm_output = getattr(resp.choices[0].message, "content", "").strip()

        # 🧩 Split into evaluation and critical notes (if present)
        evaluation_part, notes_part = None, None
        if "Critical Notes:" in llm_output:
            parts = llm_output.split("Critical Notes:", 1)
            evaluation_part = parts[0].strip()
            notes_part = parts[1].strip()
        else:
            evaluation_part = llm_output.strip()

        return {
            "evaluation_text": evaluation_part,
            "critical_notes": notes_part,
            "raw_output": llm_output
        }

    except Exception as e:
        # ⚠️ Handle any API or model-level errors
        return {"error": f"LLM call failed: {e}"}

# Factuality Evaluator
- This function uses an Azure OpenAI LLM to assess whether a generated fraud analysis output is logically consistent, factually justified, and aligned with the input claim data.
- It passes both input and output through the predefined factuality evaluation prompt and returns the LLM’s structured JSON judgment containing a score and brief explanation.

In [ ]:
def evaluate_factuality(input_text: str, output_text: str):

    # 🧩 Insert the claim input and generated output into the factuality evaluation prompt
    prompt = factuality.replace("{input}", f"{input_text}\n\nGenerated Output:\n{output_text}")

    # ⚙️ Get the Azure deployment name from environment variables
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT") or os.getenv("OPENAI_DEPLOYMENT")

    try:
        # 🚀 Call Azure OpenAI with the constructed prompt
        resp = azure_client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": prompt}],
            temperature=1.0,             # allow flexible but reasoned evaluation
            max_completion_tokens=10000, # handle long structured JSON replies
        )

        # 🧠 Extract the raw response text returned by the LLM
        llm_raw = getattr(resp.choices[0].message, "content", "").strip()

        # 🔍 Try to parse the returned JSON (factuality_score + reason)
        try:
            result = json.loads(llm_raw)
        except json.JSONDecodeError:
            # If JSON parsing fails, keep the raw LLM output for inspection
            result = {
                "factuality_score": None,
                "reason": "Invalid JSON from model",
                "raw_output": llm_raw
            }

        return result

    except Exception as e:
        # ⚠️ Handle API or network errors gracefully
        return {
            "factuality_score": None,
            "reason": f"LLM call failed: {e}"
        }

# Trajectory Evaluator
- This function compares an agent’s actual execution trajectory against the expected path stored in a reference dataset.
- It retrieves the expected steps using the given identifier, injects both trajectories into an LLM-based evaluation prompt, and returns the model’s reasoning or alignment judgment.


In [ ]:
def evaluate_trajectory(identifier: str, actual_traj: list, df):

    # 🔍 Find the matching row for the given identifier (case-insensitive match)
    row = df[df['identifier'].astype(str).str.strip().str.lower() == str(identifier).strip().lower()]
    if row.empty:
        return {"error": "identifier not found"}

    # 🧩 Find the column containing expected steps (by name pattern)
    col = next((c for c in df.columns if "expected" in c.lower() or "step" in c.lower()), None)
    if not col:
        return {"error": "No expected_steps column found"}

    # 📄 Extract the expected steps and convert the actual trajectory to a string
    expected_cell = row.iloc[0][col]
    exp_text = str(expected_cell)
    act_text = json.dumps(actual_traj, ensure_ascii=False)

    # 🧠 Build the final LLM evaluation prompt
    prompt = Trajectory.replace("{expected trajectory}", exp_text).replace("{actual trajectory}", act_text)

    # ⚙️ Retrieve Azure deployment name from environment
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT") or os.getenv("OPENAI_DEPLOYMENT")

    try:
        # 🚀 Send the evaluation request to Azure OpenAI
        resp = azure_client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": prompt}],
            temperature=1.0,
            max_completion_tokens=10000,
        )

        # 🧾 Extract the text response (handles both SDK object & dict)
        choice = resp.choices[0]
        if getattr(choice, "message", None):
            llm_raw = getattr(choice.message, "content", "") or ""
        elif isinstance(choice, dict):
            llm_raw = (choice.get("message", {}) or {}).get("content") or choice.get("text") or str(resp)
        else:
            llm_raw = str(resp)

    except Exception as e:
        # ⚠️ Return a clear error message if the LLM call fails
        return {"error": f"LLM call failed: {e}"}

    # ✅ Return both the raw model response and context for traceability
    return {
        "identifier": identifier,
        "expected_cell": expected_cell,
        "actual_traj": actual_traj,
        "llm_raw": (llm_raw or "").strip()
    }

# Path Convergence Evaluator
- This function evaluates how closely an agent’s actual reasoning path aligns with the expected design path for a given identifier.
- It retrieves the expected path from a dataset, inserts both paths into the Path Convergence LLM prompt, and returns the model’s detailed convergence analysis and score.

In [ ]:
def evaluate_path_convergence(identifier: str, actual_path: list, df):

    # 🔍 Step 1: Find the row in the dataframe that matches the given identifier (case-insensitive)
    row = df[df['identifier'].astype(str).str.strip().str.lower() == str(identifier).strip().lower()]
    if row.empty:
        return {"error": f"Identifier '{identifier}' not found"}

    # 🧩 Step 2: Detect which column contains the expected path (based on name pattern)
    col = next((c for c in df.columns if "expected" in c.lower() or "path" in c.lower()), None)
    if not col:
        return {"error": "No expected path column found"}

    # 📄 Step 3: Extract the expected path for the given identifier
    expected_path = row.iloc[0][col]

    # 🧠 Step 4: Build the prompt by inserting expected and actual paths into the Path_Convergence template
    exp_text = str(expected_path)
    act_text = json.dumps(actual_path, ensure_ascii=False)
    prompt = Path_Convergence.replace("{expected path}", exp_text).replace("{actual path}", act_text)

    # ⚙️ Step 5: Fetch the Azure OpenAI deployment name from environment variables
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT") or os.getenv("OPENAI_DEPLOYMENT")

    try:
        # 🚀 Step 6: Send the evaluation request to Azure OpenAI
        resp = azure_client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": prompt}],
            temperature=1.0,
            max_completion_tokens=10000,
        )

        # 🧾 Step 7: Extract the model's text output safely
        llm_output = (
            getattr(resp.choices[0].message, "content", "")
            if hasattr(resp.choices[0], "message")
            else str(resp)
        )

    except Exception as e:
        # ⚠️ Step 8: Return an error message if LLM request fails
        return {"error": f"LLM call failed: {e}"}

    # ✅ Step 9: Return the identifier, paths, and LLM's evaluation output
    return {
        "identifier": identifier,
        "expected_path": expected_path,
        "actual_path": actual_path,
        "llm_output": (llm_output or "").strip(),
    }

# Evaluation Pipeline

### Fetch runs
- Fetch runs from the project within a lookback window and store them in runs_sorted.
- Change LOOKBACK_HOURS and PROJECT_NAME as needed.

In [ ]:
PROJECT_NAME = "Claims processing_Sprint#1_AI ASSURANCE"
LOOKBACK_HOURS = 2   # edit if you want a different window

now = datetime.now(timezone.utc)
start_time = now - timedelta(hours=LOOKBACK_HOURS)

runs = list(client.list_runs(project_name=PROJECT_NAME, start_time=start_time, end_time=now))

# Sort by start time (latest first)
runs_sorted = sorted(runs, key=lambda x: x.start_time or x.created_at, reverse=True)

print(f"Fetched {len(runs_sorted)} runs from project '{PROJECT_NAME}' in the last {LOOKBACK_HOURS} hour(s).")

Fetched 0 runs from project 'Claims processing_Sprint#1_AI ASSURANCE' in the last 2 hour(s).


# Build unique trace list (preserve order)

Grouping spans by trace_id while preserving recency order and store trace IDs in trace_ids_list.

In [ ]:

unique_traces = []
seen_trace_ids = set()

for run in runs_sorted:
    if run.trace_id and run.trace_id not in seen_trace_ids:
        seen_trace_ids.add(run.trace_id)
        unique_traces.append(run)

trace_ids_list = [r.trace_id for r in unique_traces]
print(f"Prepared {len(trace_ids_list)} unique trace IDs.")

Prepared 0 unique trace IDs.


### Process each trace and run evaluators

For each trace, fetch spans, extract identifier from the InputAnalysis span, build the actual trajectory (tool/llm spans), and run summarizer, factuality, trajectory, and convergence evaluations where applicable. Results are printed.

In [ ]:
for trace_id in trace_ids_list:
    print(f"\n🔍 Processing Trace ID: {trace_id}")

    # Fetch all spans for this trace
    spans = list(client.list_runs(trace_id=trace_id))
    predict_traj_list = []
    identifier = None

    # Store all factuality results per span
    factuality_results = {}

    for span in spans:
        # Extract identifier from InputAnalysis span (if present)
        if span.name == "InputAnalysis":
            try:
                identifier = span.inputs.get("claim_input", {}).get("identifier")
            except Exception:
                identifier = None

        # Keep track of tool and LLM spans for trajectory
        if (span.run_type or "").lower() in ["tool", "llm"]:
            predict_traj_list.append(span.name)

            # -------------------------------
            # 🧩 Summarizer Evaluation
            # -------------------------------
            if span.name == "SummarizerRun":
                print("-> SummarizerRun: running summarizer eval")
                try:
                    score = evaluate_summarizer_accuracy(str(span.inputs), span.outputs.get("summary", ""))
                except Exception as e:
                    score = {"error": str(e)}
                print("Summarizer score:")
                print(score.get('evaluation_text', ''))
                print(score.get('critical_notes', ''))

            # -------------------------------
            # 🧠 Factuality Evaluations (All Fraud Checks)
            # -------------------------------
            if span.name in [
                "FraudDuplicateClaimCheck",
                "FraudInconsistencyCheck",
                "FraudProviderCheck",
                "FraudServiceReasonabilityCheck"
            ] and "generations" in (span.outputs or {}):
                print(f"-> {span.name}: running factuality eval")
                try:
                    gen_text = (
                        span.outputs["generations"][0][0]["text"]
                        .strip("```json")
                        .strip("```")
                    )
                    factual_res = evaluate_factuality(span.inputs, gen_text)
                except Exception as e:
                    factual_res = {"error": str(e)}

                # ✅ Store result under this specific span name
                factuality_results[span.name] = factual_res
                print(f"   factuality score for {span.name}:", factual_res)

    # -------------------------------
    # 🔁 Reverse trajectory for chronological order
    # -------------------------------
    traj = predict_traj_list[::-1]

    # -------------------------------
    # 🧭 Trajectory Evaluation
    # -------------------------------
    try:
        traj_result = evaluate_trajectory(identifier, traj, df)
    except Exception as e:
        traj_result = {"error": str(e)}
    print("\ntrajectory result:")
    print(traj_result.get('llm_raw', traj_result))

    # -------------------------------
    # 🔗 Path Convergence Evaluation
    # -------------------------------
    try:
        conv_result = evaluate_path_convergence(identifier, traj, df)
    except Exception as e:
        conv_result = {"error": str(e)}
    print("\nconvergence result:")
    print(conv_result.get('llm_output', conv_result))

    # -------------------------------
    # 🗂️ Optional: Print summary of all factual results
    # -------------------------------
    if factuality_results:
        print("\n📊 Summary of all factuality evaluations:")
        for span_name, result in factuality_results.items():
            print(f" - {span_name}: {result}")

### Log it back to Langsmith

In [ ]:
import re

# ---------------------------
# 🔧 Helper: Extract numeric score from text
# ---------------------------
def extract_score(text, key_patterns):
    """Extract numeric values (floats) from evaluator text using regex patterns."""
    if not text:
        return None
    for pat in key_patterns:
        match = re.search(pat, text, flags=re.IGNORECASE)
        if match:
            try:
                return float(match.group(1))
            except Exception:
                pass
    return None


# ---------------------------
# 📝 Iterate again to log results to LangSmith
# ---------------------------
for trace_id in trace_ids_list:
    print(f"\n🪶 Logging feedback for Trace ID: {trace_id}")

    # Re-fetch spans
    spans = list(client.list_runs(trace_id=trace_id))
    span_ids = {s.name: getattr(s, "id", None) for s in spans}

    # --- 1️⃣ Summarizer Accuracy Feedback ---
    if "score" in locals() and isinstance(score, dict):
        eval_text = score.get("evaluation_text") or ""
        notes = score.get("critical_notes") or ""
        summ_score = extract_score(eval_text, [r"Final Accuracy Score[:\- ]+([0-9]+(?:\.[0-9]+)?)"])
        try:
            client.create_feedback(
                key="SummarizerAccuracy",
                score=float(summ_score) if summ_score is not None else None,
                run_id=span_ids.get("SummarizerRun"),
                trace_id=trace_id,
                comment=(eval_text + "\n\n" + notes)[:4000],
            )
            print(f"✅ Logged SummarizerAccuracy → Span: {span_ids.get('SummarizerRun')}, Score: {summ_score}")
        except Exception as e:
            print("⚠️ Summarizer logging failed:", e)

    # --- 2️⃣ Factuality Feedback (all four Fraud checks) ---
    if "factuality_results" in locals() and isinstance(factuality_results, dict):
        for span_name, factual_res in factuality_results.items():
            span_id = span_ids.get(span_name)
            factual_score = factual_res.get("factuality_score")
            factual_reason = factual_res.get("reason") or str(factual_res)
            try:
                client.create_feedback(
                    key=f"Factuality_{span_name}",
                    score=float(factual_score) if factual_score is not None else None,
                    run_id=span_id,
                    trace_id=trace_id,
                    comment=factual_reason[:4000],
                )
                print(f"✅ Logged {span_name} Factuality → Span: {span_id}, Score: {factual_score}")
            except Exception as e:
                print(f"⚠️ Logging failed for {span_name}:", e)

    # --- 3️⃣ Trajectory Feedback ---
    traj_text = traj_result.get("llm_raw") if isinstance(traj_result, dict) else ""
    traj_score = extract_score(traj_text, [r"Trajectory Score[:\- ]+([0-9]+(?:\.[0-9]+)?)"])
    try:
        client.create_feedback(
            key="Trajectory",
            score=float(traj_score) if traj_score is not None else None,
            run_id=None,  # trace-level
            trace_id=trace_id,
            comment=traj_text[:4000],
        )
        print(f"✅ Logged Trajectory → Trace: {trace_id}, Score: {traj_score}")
    except Exception as e:
        print("⚠️ Trajectory logging failed:", e)

    # --- 4️⃣ Path Convergence Feedback ---
    if isinstance(conv_result, dict):
        conv_output = conv_result.get("llm_output", "")
        conv_score = extract_score(conv_output, [
            r"Final Path Convergence Score[:\- ]+([0-9]+(?:\.[0-9]+)?)",
            r"Convergence Score[:\- ]+([0-9]+(?:\.[0-9]+)?)"
        ])
        try:
            client.create_feedback(
                key="PathConvergence",
                score=float(conv_score) if conv_score is not None else None,
                run_id=None,  # trace-level
                trace_id=trace_id,
                comment=(conv_output or "")[:4000],
            )
            print(f"✅ Logged PathConvergence → Trace: {trace_id}, Score: {conv_score}")
        except Exception as e:
            print("⚠️ Path Convergence logging failed:", e)